In [1]:
!git clone https://github.com/harshavardhan-patil/mimolm.git

Cloning into 'mimolm'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 187 (delta 84), reused 156 (delta 53), pack-reused 0 (from 0)
Receiving objects: 100% (187/187), 317.96 KiB | 3.53 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [2]:
%cd mimolm
!git checkout feature/sampling

/content/mimolm
Branch 'feature/sampling' set up to track remote branch 'feature/sampling' from 'origin'.
Switched to a new branch 'feature/sampling'


In [3]:
%pip install lightning
%pip install python-dotenv
%pip install loguru
#%pip install nvidia-dali-cuda120
%pip install transforms3d
%pip install omegaconf
%pip install tensordict

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 6.6 MB/s eta 0:00:00


In [2]:
!cp /content/drive/MyDrive/train.h5 /content/mimolm/data

In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive')
# Append the directory to your python path using sys
sys.path.append('/content/mimolm')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from src.external.hptr.src.data_modules.data_h5_av2 import DataH5av2
from src.mimolm import MimoLM
import lightning as pl
import time
import torch
import tqdm

data_module = DataH5av2("/content/mimolm/data")
data_module.setup(stage="fit")
data_loader = data_module.train_dataloader()


model = MimoLM(data_size=data_module.tensor_size_train)
trainer = pl.Trainer(precision='16-mixed', max_epochs=2, profiler="simple", default_root_dir="/content/drive/MyDrive/Colab/mimolm/ckpts")
trainer.fit(model=model, train_dataloaders=data_loader)

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name              | Type               | Params | Mode 
-----------------------------------------------------------------
0 | preprocessor      | Sequential         | 0      | train
1 | input_projections | InputProjections   | 428 K  | train
2 | encoder           | EarlyFusionEncoder | 2.4 M  | train
3 | decoder           | MotionDecoder   

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=2` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.
INFO: FIT Profiler Report

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                        	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                              

In [ ]:
# # enable grads
# torch.set_grad_enabled(True)

# losses = []
# optimizer = model.configure_optimizers()
# start = time.perf_counter()
# c1 = 0
# c2 = 0
# c3 = 0
# c4 = 0
# c5 = 0
# transfer = 0
# training = 0
# backward = 0
# optim = 0
# for batch in data_loader:
#     c0 = time.perf_counter()
#     batch = model.on_before_batch_transfer(batch, 0).to('cuda')
#     c1 = time.perf_counter()
#     # train step
#     loss = model.training_step(batch)
#     c2 = time.perf_counter()
#     # clear gradients
#     optimizer.zero_grad()
#     c3 = time.perf_counter()
#     # backward
#     loss.backward()
#     c4 = time.perf_counter()

#     # update parameters
#     optimizer.step()
#     c5 = time.perf_counter()

#     transfer += c1 - c0
#     training += c2 - c1
#     backward += c4 - c3
#     optim += c5 - c4
#     losses.append(loss)


# end = time.perf_counter()